<a href="https://colab.research.google.com/gist/raissaofernandes/f91986a0f236c9e074241ae67cd66db0/tutorial-pre_processamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tutorial de pré-processamento de dados utilizando Python**
> Python é uma linguagem de programação bastante utilizada em Processamento de Linguagem Natural. Um passo importante para a análise é o pré-processamento de dados da sua base de treinamento e teste.



###Primeiramente, vamos importar algumas bibliotecas que serão usadas.

In [1]:
#Ao longo do processo irei explicar o intuito e a função de cada.
import nltk
import unicodedata
import pandas as pd 
import re
nltk.download('punkt')
from nltk import word_tokenize
import tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


###Em seguida, precisaremos carregar o dataset. Para isso, usaremos a biblioteca Pandas.

In [2]:
#Para um conjunto de dados para treinamento e teste é importante que o conjunto esteja claro e de acordo com o objeto de pesquisa;
#Aqui, por se tratar de um tutorial rápido, o conjundo de dados foi escolhido por não apresentar um volume muito grande de dados e 
#por possuir informações objetivas;
balneabilidade_bahia = pd.read_csv('https://raw.githubusercontent.com/raissaofernandes/balneabilidade-bahia/master/balneabilidade-bahia.csv')
balneabilidade_bahia.head(5)

,id_campanha,numero_boletim,data_emissao,costa_ponto,local_da_coleta,categoria,ponto_codigo,tipo_campanha
0,6281,9/2013,2013-02-25,SSA,"Ao lado do forte Monte Serrat, em frente ao mu...",Própria,Boa Viagem - SSA BV 100,CAMPANHA
1,6281,9/2013,2013-02-25,SSA,"Atrás das antigas instalações da FIB, Rua Agrá...",Própria,Canta Galo - SSA CG 100,CAMPANHA
2,6281,9/2013,2013-02-25,SSA,"Atrás do Hospital Sagrada Familia, em frente à...",Imprópria,Pedra Furada - SSA FU 100,CAMPANHA
3,6281,9/2013,2013-02-25,SSA,"Av. Sete de Setembro, em frente à Rua César Za...",Imprópria,Porto da Barra - SSA PB 100,CAMPANHA
4,6281,9/2013,2013-02-25,SSA,Em frente à barraca do Valença.,Imprópria,Penha - SSA PE 100,CAMPANHA


##Agora vamos começar efetivamente o pré-processamento dos dados.
O pré-processamento de dados é uma parte importante para facilitar a compreensão da máquina e o aprendizado no uso do seu conjunto de dados.

> Para seguir precisamos entender um pouco sobre o que são as expressões regulares. Existe uma seção única na [documentação](https://docs.python.org/3/library/re.html) do Python explicando detalhadamente cada uma delas. Recomendo a leitura.



Brevemente, expressões regulares (chamadas REs, na sigla em Inglês) são essencialmente uma pequena linguagem de programação altamente especializada incorporada no Python e disponibilizada através do módulo re. Por definição, as expressões regulares são uma sequência de caracteres que te ajudam a juntar ou achar outras strings, ou até mesmo uma sequência de strings. 

### Limpeza de dados
Inicialmente, vamos limpar os dados para deixá-los mais enxutos e mais fáceis de serem visualizados.

Caso fique alguma dúvida sobre os métodos, recomendo fortemente dar uma lida na [documentação](https://docs.python.org/pt-br/3/library/stdtypes.html#str.lower) do Python. É uma prática muito boa para quem está começando a aprender.


In [3]:
#A função limpeza_dados recebe dois parâmetros;
#o parâmetro dataset corresponde ao conjunto de dados;
#a coluna_texto especifica qual coluna do dataset será limpa.
def limpeza_dados(dataset, coluna_texto): 
    dataset[coluna_texto] = dataset[coluna_texto].str.lower() #O método lower() retorna uma cópia da string com todos os caracteres convertidos para letras minúsculas.
    #O método replace() retorna uma cópia da string com todas as ocorrências da substring anterior substituídas por uma nova:
    dataset[coluna_texto] = dataset[coluna_texto].str.replace(r"#", " ") #nesse caso, a (#) foi substituida por um espaço em branco (" ");
    dataset[coluna_texto] = dataset[coluna_texto].str.replace(r"http", " ") #nesse caso, a marcação (http) foi substituido por um espaço em branco (" ");
    dataset[coluna_texto] = dataset[coluna_texto].str.replace("-", " ") #nesse caso, o (-) foi substituido por um espaço em branco (" ");
    dataset[coluna_texto] = dataset[coluna_texto].str.replace("\n", " ") #nesse caso, a quebra de linha (\n) foi substituida por um espaço em branco (" ").
    return dataset #Em seguida retornamos o dataset já com as colunas de textos limpas. 

#Chamaremos a função para limpar nossos dados que estão no dataset declarado anteriormente.
balneabilidade_bahia = limpeza_dados(balneabilidade_bahia, 'local_da_coleta') #Nesse caso, o dataset está na variável balneabilidade_bahia, a coluna é a local_da_coleta.
balneabilidade_bahia = limpeza_dados(balneabilidade_bahia, 'ponto_codigo') #Nesse caso, o dataset está na variável balneabilidade_bahia, a coluna é a ponto_codigo.
balneabilidade_bahia = limpeza_dados(balneabilidade_bahia, 'categoria') #Nesse caso, o dataset está na variável balneabilidade_bahia, a coluna é a categoria.
balneabilidade_bahia = limpeza_dados(balneabilidade_bahia, 'tipo_campanha') #Nesse caso, o dataset está na variável balneabilidade_bahia, a coluna é a tipo_campanha.
balneabilidade_bahia = limpeza_dados(balneabilidade_bahia, 'costa_ponto') #Nesse caso, o dataset está na variável balneabilidade_bahia, a coluna é a costa_ponto.

In [4]:
#Agora, iremos ler os dados do nosso conjunto a fim de descobrir se nossa função deu realmente certo.
balneabilidade_bahia.head()

,id_campanha,numero_boletim,data_emissao,costa_ponto,local_da_coleta,categoria,ponto_codigo,tipo_campanha
0,6281,9/2013,2013-02-25,ssa,"ao lado do forte monte serrat, em frente ao mu...",própria,boa viagem ssa bv 100,campanha
1,6281,9/2013,2013-02-25,ssa,"atrás das antigas instalações da fib, rua agrá...",própria,canta galo ssa cg 100,campanha
2,6281,9/2013,2013-02-25,ssa,"atrás do hospital sagrada familia, em frente à...",imprópria,pedra furada ssa fu 100,campanha
3,6281,9/2013,2013-02-25,ssa,"av. sete de setembro, em frente à rua césar za...",imprópria,porto da barra ssa pb 100,campanha
4,6281,9/2013,2013-02-25,ssa,em frente à barraca do valença.,imprópria,penha ssa pe 100,campanha


**Veja que nossa função apresentou os resultados esperados. Com isso, seguiremos para o próximo passo do pré-processamento de dados.**

### Remover caracteres especiais e acentos
Outra função que criaremos é a de remover caracteres especiais e os acentos das palavras. 

In [5]:
#Para a função caracteres_especiais receberemos um único parâmetro palavra.
def caracteres_especiais(palavra):
    #A variável nfkd foi criada para que possamos atribuir ao unicodedata sua forma normal novamente;
    #isto é, o unicode é uma forma de fornecer um número único para cada caractere, basicamente é um padrão utilizado mundialmente;
    #unicodedata.normalize('NFKD', palavra) retorna a forma normal NFKD para o unicode string palavra;
    #essa normalização foi guardada na variável nfkd para que possamos utilizar posteriormente.
    nfkd = unicodedata.normalize('NFKD', palavra)
    #A variável palavraSemAcento será utilizada para o unicodedata.combining(c) retornar a canonical combining class[ccc], assinando o caractere como um inteiro;
    #a grosso modo, cada caractere Unicode tem uma classe combinada associada a ele;
    #utilizar o unicodedata.combining(c) permite que o valor 0 seja retornado caso não haja uma classe combinada definida;
    #a ccc de um caracter é um número no intervalo 0..255.
    palavra_sem_acento = u"".join([c for c in nfkd if not unicodedata.combining(c)]) #.join serve para construir uma lista de strings que é a concatenação das strings passadas no método.
    #Agora a variável palavra_sem_acento removerá definitivamente os caracteres especiais e acentos das palavras;
    #logo no início importamos a biblioteca re para utilizá-la agora;
    #a biblioteca re contém as expressões regulares que serão removidas.
    palavra_sem_acento = re.sub('[^a-zA-Z0-9 \\\]', '', palavra_sem_acento)
    #re.sub('[^a-zA-Z0-9 \\\]',...) reescreve a string obtida depois de ter substituído pelo menos a maioria das ocorrências padrão de [^a-zA-Z0-9 \\\] na 
    #string palavra_sem_acento;
    #re.sub(...,'',...) é o que será nosso substituto.
    return palavra_sem_acento #Agora, finalmente, damos o retorno da nossa função com a palavra_sem_acento.

In [6]:
#Chamaremos a função para remover os caracteres especiais.
#Nesse caso, o dataset é balneabilidade_bahia, a coluna especificada é a local_da_coleta.
balneabilidade_bahia['local_da_coleta'] = [caracteres_especiais(str(t)) for t in balneabilidade_bahia['local_da_coleta']]
#Nesse caso, o dataset é balneabilidade_bahia, a coluna especificada é a categoria.
balneabilidade_bahia['categoria'] = [caracteres_especiais(str(t)) for t in balneabilidade_bahia['categoria']]

In [7]:
#Agora vamos ver novamente os dados do nosso conjunto a fim de descobrir se nossa função deu realmente certo.
balneabilidade_bahia.head(5)

,id_campanha,numero_boletim,data_emissao,costa_ponto,local_da_coleta,categoria,ponto_codigo,tipo_campanha
0,6281,9/2013,2013-02-25,ssa,ao lado do forte monte serrat em frente ao mur...,propria,boa viagem ssa bv 100,campanha
1,6281,9/2013,2013-02-25,ssa,atras das antigas instalacoes da fib rua agrar...,propria,canta galo ssa cg 100,campanha
2,6281,9/2013,2013-02-25,ssa,atras do hospital sagrada familia em frente a ...,impropria,pedra furada ssa fu 100,campanha
3,6281,9/2013,2013-02-25,ssa,av sete de setembro em frente a rua cesar zama...,impropria,porto da barra ssa pb 100,campanha
4,6281,9/2013,2013-02-25,ssa,em frente a barraca do valenca,impropria,penha ssa pe 100,campanha


**Veja que nossa função apresentou os resultados esperados. Com isso, seguiremos para o próximo passo do pré-processamento de dados.**

### Remover stopwords
Uma função indispensável no pré-processamento de dados para NPL é a remoção das stopwords. Considera-se uma stopword qualquer palavra que seja irrelevante numa busca e que não tire o sentido do texto caso seja removida.

In [8]:
#Declaração das stopwords que serão retiradas quando criarmos nossa função.
stopwords = ['pra', 'pro','to', 'ta','de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para', 'a', 'com', 'nao', 'uma', 'os', 'no', 'se', 'a', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'foi', 'ao', 'ele', 'das', 'tem', 'a', 'seu', 'sua', 'ou', 'ser', 'quando', 'muito', 'ha', 'nos', 'ja', 'esta', 'eu', 'tambam', 'sa3', 'pelo', 'pela', 'ata', 'isso', 'ela', 'entre', 'era', 'depois', 'sem', 'mesmo', 'aos', 'ter', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'estao', 'vocaa', 'tinha', 'foram', 'essa', 'num', 'nem', 'suas', 'meu', 'a s', 'minha', 'taam', 'numa', 'pelos', 'elas', 'havia', 'seja', 'qual', 'sera', 'na3s', 'tenho', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'fosse', 'dele', 'tu', 'te', 'vocaas', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 
'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'esta', 'estamos', 'estao', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estavamos', 'estavam', 'estivera', 'estivaramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivassemos', 'estivessem', 'estiver', 'estivermos', 'estiverem', 'hei', 'ha', 'havemos', 'hao', 'houve', 'houvemos', 'houveram', 'houvera', 'houvaramos', 'haja', 'hajamos', 'hajam', 'houvesse', 'houvassemos', 'houvessem', 'houver', 'houvermos', 'houverem', 'houverei', 'houvera', 'houveremos', 'houverao', 'houveria', 'houveraamos', 'houveriam', 'sou', 'somos', 'sao', 'era', 'aramos', 'eram', 'fui', 'foi', 'fomos', 'foram', 'fora', 'fa ramos', 'seja', 'sejamos', 'sejam', 'fosse', 'fa ssemos', 'fossem', 'for', 'formos', 'forem', 'serei', 'sera', 'seremos', 'serao', 'seria', 'seraamos', 'seriam', 'tenho', 'tem', 'temos', 'tam', 'tinha', 'tanhamos', 'tinham', 'tive', 'teve', 'tivemos', 'tiveram', 'tivera', 'tivaramos', 'tenha', 'tenhamos', 'tenham', 'tivesse', 'tivassemos', 'tivessem', 'tiver', 'tivermos', 'tiverem', 'terei', 'tera', 'teremos', 'terao', 'teria', 'teraamos', 'teriam']
len(stopwords) #Checando o tamanho do nosso array/quantidade de stopwords contidas nele

225

In [9]:
#Primeiramente, definiremos nossa função remove_stopWords que receberá um único parâmetro coluna_texto.
def remove_stopwords(coluna_texto):
    frase = [] #Criaremos um array frase para ser usado posteriormente;
    #em Py, o método split retorna uma lista de strings depois de quebrar a string dada pelo separador especificado;
    #nesse caso, não especificamos nenhum separador  porque o intuito é que ele separe as palavras separadas por um espaço em branco.
    for word in coluna_texto.split():
        if word not in stopwords: #Caso a palavra (word) não seja nenhuma das stopwords declaradas, faremos:
            frase.append(word)    #Adicionaremos ao nosso array frase o item word, por isso utilizamos o método append.
    #Separaremos os itens do nosso array por um espaço em branco (' ') e utilizaremos o .join que já teve sua utilidade explicada na função de caracteres_especiais;
    return ' '.join(frase) #Retornaremos nossos valores com as stopwords removidas.       

In [10]:
#Chamaremos a função para remover as stopwords;
#aqui o dataset é balneabilidade_bahia, a coluna especificada é a local_da_coleta.
balneabilidade_bahia['local_da_coleta'] = [remove_stopwords(str(t)) for t in balneabilidade_bahia['local_da_coleta']]
#Vamos observar os valores do nosso conjunto de dados.
balneabilidade_bahia
#Repare que mesmo após as stopwords terem sido removidas as coordenadas não deixaram de fazer sentido;
#isso é possível justamento pelas stopwords serem utilizadas para ligar palavras e não necessariamente para dar sentido a elas.

,id_campanha,numero_boletim,data_emissao,costa_ponto,local_da_coleta,categoria,ponto_codigo,tipo_campanha
0,6281,9/2013,2013-02-25,ssa,lado forte monte serrat frente muro lateral fu...,propria,boa viagem ssa bv 100,campanha
1,6281,9/2013,2013-02-25,ssa,atras antigas instalacoes fib rua agrario mene...,propria,canta galo ssa cg 100,campanha
2,6281,9/2013,2013-02-25,ssa,atras hospital sagrada familia frente ladeira ...,impropria,pedra furada ssa fu 100,campanha
3,6281,9/2013,2013-02-25,ssa,av sete setembro frente rua cesar zama junto e...,impropria,porto da barra ssa pb 100,campanha
4,6281,9/2013,2013-02-25,ssa,frente barraca valenca,impropria,penha ssa pe 100,campanha
5,6281,9/2013,2013-02-25,ssa,frente casa vila maria lado rampa acesso praia,propria,são tomé de paripe ssa in 100,campanha
6,6281,9/2013,2013-02-25,ssa,frente escada acesso praia frente portinox rua...,impropria,pituba ssa pi 100,campanha
7,6281,9/2013,2013-02-25,ssa,frente colegio pm antigo colegio joao florenci...,impropria,bogari ssa bo 100,campanha
8,6281,9/2013,2013-02-25,ssa,frente hotel alah mar rua joao mendes costa,impropria,armação ssa ar 200,campanha
9,6281,9/2013,2013-02-25,ssa,frente hotel grande stella maris,propria,stella mares ssa st 100,campanha
